In [67]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
block_size=8
batch_size=4
max_iters=1000
learning_rate=3e-4
eval_iters=250
#dropout=0.2

cpu


In [68]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
chars = sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [69]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [string_to_int[c] for c in s]
decode=lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0, 26, 49,  0, 36, 11,  1,
        30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0, 25, 45, 44, 32, 39, 42,  1,
        39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39,
        50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,  1, 39, 50,  9,
         1, 39, 50, 37, 25,  1, 39, 30,  1, 39])


In [70]:
n = int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data=train_data if split == 'train' else val_data
    ix= torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device), y.to(device)
    return x,y

x,y=get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[73, 65, 78,  1, 72, 73, 54, 73],
        [62, 57,  1, 73, 61, 58,  1, 40],
        [ 0, 72, 76, 68, 71, 57,  1, 54],
        [ 1, 68, 67, 58,  1, 73, 61, 58]])
targets: 
tensor([[65, 78,  1, 72, 73, 54, 73, 58],
        [57,  1, 73, 61, 58,  1, 40, 71],
        [72, 76, 68, 71, 57,  1, 54, 67],
        [68, 67, 58,  1, 73, 61, 58, 78]])


In [71]:
block_size=8

x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print("when input is",context,"target is",target)

when input is tensor([80]) target is tensor(28)
when input is tensor([80, 28]) target is tensor(39)
when input is tensor([80, 28, 39]) target is tensor(42)
when input is tensor([80, 28, 39, 42]) target is tensor(39)
when input is tensor([80, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(49)
when input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(1)


In [65]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [62]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embeddings_table=nn.Embedding(vocab_size,vocab_size)

    def forward (self,index,targets=None):
        logits=self.token_embeddings_table(index)

        if targets is None:
            loss=None
        else:
            B,T,C = logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
            
        return logits, loss
        
    def generate(self,index,max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss=self.forward(index)
            logits=logits[:,-1,:]
            probs=F.softmax(logits,dim=-1)
            index_next= torch.multinomial(probs,num_samples=1)
            index=torch.cat((index,index_next),dim=1)
        return index

model=BigramLanguageModel(vocab_size)
m=model.to(device)

context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)
            


X(n:VOs2FjGEdK&Y'mHTnObAJH5mI&J9'GO7'QMU
﻿_mK!9O29;d7:Im﻿s,ZI&YJ( ;)Gf3[x
sc4oeo[2MIRsk?v&nqmv.DI0!8(K&;t0(Hb[n
vKEd8gdray[mT﻿f':-f3"SVDAP1h0n)25Um:f*6Tn_u_!Qs6X*a!ft]U,d0﻿fPph:5GfRa &m,n_i(hu4cP]wGCesRJ?c75RqUo'.Au6
HK!g?v&NLHx?J?wPR,[t!?w6r3e]1hjOfTNv5&GptK&e7N3Q(gHCQy:5:_VMi8vZoo*O6a?'[flj8R
RWe-*YkqHCa"qL&ieU81owNir[&*aY!NnYdRL&
Zi(F
8w?EweL-VH5]viHmFV(b&N)qdQlfRLGcb﻿AWf1-RYjnCv!5Zz[GG[T:VRo[F.[Hrau"xu6!&?&i6"1Z-'ZY'Or3QR1]!9iC8q
Q1N87mxxxEr,fRAj"YaC&YaSRF6kJ:Ha(6RJL2;dbY0 kl]&Yy1VB4V**!fV,?


In [74]:
optimizer= torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter%eval_iters==0:
        losses=estimate_loss()
        print(f"step:{iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    xb,yb=get_batch('train')
    logits,loss=model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step:0, train loss: 2.846, val loss: 2.904
step:250, train loss: 2.839, val loss: 2.846
step:500, train loss: 2.816, val loss: 2.849
step:750, train loss: 2.811, val loss: 2.866
2.9209296703338623


In [54]:
context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


A:httttinoo&Zkigowene  stoX!]yfit'rerouthO) siveero
"Wir, ackas,

e'SHHwe a-, t_J-ge ar inyos m;qhC)Mq

aily t kmas tie MRntthingn,tu, s
IBhe  einsedm, wes?OOhof7R_N6"Ymok u'I.]!rn ot ul, ant  t(;THWP)Suglon blit Uv73ingrm;4MRKP5
elo tcR s ged'S7]un buX3P&S7n
kx(;pan hUCXB0ENDmowearsathe s'j?hepiZtede y  ef i. Gn:&﻿:7S7RAegrys mount,"D]y GMRTH8JRyengoya, thugn a;x3u
"

X  v'te t e;Y59clavandBLJV']
1" an"_v5MVz&beav,7Vwigl
A ss G1r
Awouims IGENat s L1HI,"NngJMYINTmed Cto, t

"wan&Mf s a')3N&8Gsex
